In [1]:
using CSV
using DataFrames

include("functions.jl")
using Random

In [2]:
using ScikitLearn

@sk_import svm:SVC;
@sk_import tree:DecisionTreeClassifier;
@sk_import ensemble:VotingClassifier
@sk_import neighbors: KNeighborsClassifier;


## Read Data

In [3]:
file_path = "dataset/super_simplified_Android_Malware.csv"

data = CSV.File(file_path, header=true) |> DataFrame;

In [4]:
describe(data, :all)

Row,variable,mean,std,min,q25,median,q75,max,sum,nunique,nuniqueall,nmissing,nnonmissing,first,last,eltype
,Symbol,Union…,Union…,Any,Union…,Union…,Union…,Any,Union…,Union…,Int64,Int64,Int64,Any,Any,DataType
1,Flow ID,,,1.31.173.21-10.42.0.151-80-36854-6,,,,8.8.8.8-10.42.0.211-53-3181-17,,3502,3502,0,3557,157.240.0.36-10.42.0.211-443-55364-6,10.42.0.211-10.42.0.1-3890-53-17,String
2,Source IP,,,0.0.0.0,,,,96.6.164.184,,246,246,0,3557,10.42.0.211,10.42.0.211,String15
3,Source Port,39310.9,17860.1,0,34948.0,43520.0,52110.0,65400,139828814,,3013,0,3557,55364,3890,Int64
4,Destination IP,,,1.31.173.21,,,,98.139.225.43,,826,826,0,3557,157.240.0.36,10.42.0.1,String15
5,Destination Port,5390.09,14590.5,0.0,80.0,443.0,443.0,60729.0,1.91725e7,,400,0,3557,443.0,53.0,Float64
6,Protocol,8.26061,4.51586,0.0,6.0,6.0,6.0,17.0,29383.0,,3,0,3557,6.0,17.0,Float64
7,Timestamp,,,04/07/2017 10:08:16,,,,30/06/2017 12:59:10,,3332,3332,0,3557,26/06/2017 12:43:14,13/06/2017 08:39:01,String31
8,Flow Duration,1.11761e7,2.19812e7,2,48777.0,557126.0,1.08321e7,119977227,39753215647,,3442,0,3557,65319091,48681,Int64
9,Total Fwd Packets,7.5932,59.0075,1,1.0,2.0,5.0,3246,27009,,97,0,3557,9,1,Int64


In [5]:
import StatsBase: countmap

columns_to_drop = ["Flow ID", " Timestamp"]
columns = names(data)

println("Size of dataframe before dropping columns $(size(data))")
for column in 1:size(data, 2)
    unique_values = countmap(data[:, column])

    if length(unique_values) == 1
        println("Adding column $(columns[column])")
        # println(unique_values)
        push!(columns_to_drop, columns[column])
    end
    
end

select!(data, Not(columns_to_drop))

println("Size of dataframe after dropping columns $(size(data))")

dropmissing!(data)

println("Size of dataframe after dropping nulls $(size(data))")

unique_data = unique(data)

println("Size of dataframe after dropping duplicating rows $(size(data))")

Size of dataframe before dropping columns (3557, 85)
Adding column  Bwd PSH Flags
Adding column  Fwd URG Flags
Adding column  Bwd URG Flags
Adding column  RST Flag Count
Adding column  CWE Flag Count
Adding column  ECE Flag Count
Adding column Fwd Avg Bytes/Bulk
Adding column  Fwd Avg Packets/Bulk
Adding column  Fwd Avg Bulk Rate
Adding column  Bwd Avg Bytes/Bulk
Adding column  Bwd Avg Packets/Bulk
Adding column Bwd Avg Bulk Rate
Size of dataframe after dropping columns (3557, 71)
Size of dataframe after dropping nulls (3557, 71)
Size of dataframe after dropping duplicating rows (3557, 71)


In [6]:
function ip_to_decimal(ip)
    # Split the IP address into octets
    octets = split(ip, '.')
    # Convert each octet to binary and combine them into a single 32-bit number
    binary = join([string(parse(Int, octet, base=10), base=2, pad=8) for octet in octets])
    decimal = parse(Int, binary, base=2) # Convert binary to decimal
    return decimal
end

source_ips = data[!, :" Source IP"];
destination_ips = data[!, :" Destination IP"];

data[!, :"Source IP Decimal"] = map(ip -> ip_to_decimal(ip), source_ips);
data[!, :"Destination IP Decimal"] = map(ip -> ip_to_decimal(ip), destination_ips);

select!(data, Not([" Source IP", " Destination IP"]));


In [7]:
describe(data)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,Source Port,39310.9,0,43520.0,65400,0,Int64
2,Destination Port,5390.09,0.0,443.0,60729.0,0,Float64
3,Protocol,8.26061,0.0,6.0,17.0,0,Float64
4,Flow Duration,1.11761e7,2,557126.0,119977227,0,Int64
5,Total Fwd Packets,7.5932,1,2.0,3246,0,Int64
6,Total Backward Packets,11.0748,0,1.0,8452,0,Int64
7,Total Length of Fwd Packets,665.05,0.0,31.0,110678.0,0,Float64
8,Total Length of Bwd Packets,12084.5,0.0,17.0,1.22247e7,0,Float64
9,Fwd Packet Length Max,218.112,0.0,31.0,1460.0,0,Float64


## First approach: binary classification

In [8]:
function transform_binary_class(output_data)
    binary_labels = output_data .!= "Benign"
    return binary_labels
end

output_data = data[!, :Label];
select!(data, Not(:Label))

input_data = Matrix(data[!, 1:size(data, 2)]);

binary_labels = transform_binary_class(output_data)
@assert binary_labels isa BitVector
@assert input_data isa Matrix

### Preprocessing

In [9]:
Random.seed!(42)

train_indexes, test_indexes = holdOut(size(input_data, 1), 0.2)

train_input = convert(Array{Float32, 2}, input_data[train_indexes, :])
train_output = binary_labels[train_indexes]

normalizationParameters = calculateMinMaxNormalizationParameters(train_input)

normalizeMinMax!(train_input, normalizationParameters)

test_input = convert(Array{Float32, 2}, input_data[test_indexes, :])
test_output = binary_labels[test_indexes]

normalizeMinMax!(test_input, normalizationParameters)

@assert size(test_input, 1) == size(test_output, 1)
@assert size(train_input, 1) == size(train_output, 1)

In [10]:
Random.seed!(42)

kFolds = 10
crossValidationIndexes = crossvalidation(train_output, kFolds);

In [11]:
function generate_latex_table(metrics::Dict{String, <: Any}, final::Bool)
    
    topology = metrics["topology"]
    accuracy = metrics["accuracy"]
    recall = metrics["recall"]
    specificity = metrics["specificity"]
    f1_score = metrics["f1_score"]
    
    if final
        confusion_matrix = metrics["confusion_matrix"]
        println("$topology & $(round(accuracy*100, digits=2))\\%  & $(round(recall*100, digits=2))\\%  & $(round(specificity*100, digits=2))\\%  & $(round(f1_score*100, digits=2))\\% & $confusion_matrix \\\\")
    else
        std_accuracy = metrics["std_accuracy"]
        std_recall = metrics["std_recall"]
        std_specificity = metrics["std_specificity"]
        std_f1_score = metrics["std_f1_score"]
        println("$topology & $(round(accuracy*100, digits=2))\\% \\textit{($(round(std_accuracy, digits = 2)))} & $(round(recall*100, digits=2))\\% \\textit{($(round(std_recall, digits = 2)))} & $(round(specificity*100, digits=2))\\% \\textit{($(round(std_specificity, digits = 2)))} & $(round(f1_score*100, digits=2))\\% \\textit{($(round(std_f1_score, digits = 2)))} \\\\")
    end
    
end

generate_latex_table (generic function with 1 method)

### kNN

In [12]:
include("functions.jl")
knnParameters = Dict("modelType" => :kNN, "numNeighboors" => 0)

ks = [3 , 5, 7, 10, 15, 20]
for k in ks
    knnParameters["numNeighboors"] = k
    metricsCV = (modelCrossValidation(knnParameters["modelType"], knnParameters, train_input, train_output, crossValidationIndexes))
    metricsCV["topology"] = k

    generate_latex_table(metricsCV, false)
end

println("----------------------------------------------------------------")
for k in ks
    knnParameters["numNeighboors"] = k
    metrics = createAndTrainFinalModel(knnParameters["modelType"], knnParameters, train_input, train_output, test_input, test_output)
    metrics["topology"] = k

    generate_latex_table(metrics, true)
end

3 & 92.79\% \textit{(0.01)} & 98.95\% \textit{(0.01)} & 1.14\% \textit{(0.02)} & 96.26\% \textit{(0.0)} \\
5 & 93.43\% \textit{(0.0)} & 99.7\% \textit{(0.0)} & 0.0\% \textit{(0.0)} & 96.6\% \textit{(0.0)} \\
7 & 93.57\% \textit{(0.0)} & 99.85\% \textit{(0.0)} & 0.0\% \textit{(0.0)} & 96.68\% \textit{(0.0)} \\
10 & 93.67\% \textit{(0.0)} & 99.96\% \textit{(0.0)} & 0.0\% \textit{(0.0)} & 96.73\% \textit{(0.0)} \\
15 & 93.71\% \textit{(0.0)} & 100.0\% \textit{(0.0)} & 0.0\% \textit{(0.0)} & 96.75\% \textit{(0.0)} \\
20 & 93.71\% \textit{(0.0)} & 100.0\% \textit{(0.0)} & 0.0\% \textit{(0.0)} & 96.75\% \textit{(0.0)} \\
----------------------------------------------------------------
3 & 90.03\%  & 97.86\%  & 1.72\%  & 94.74\% & [640 57; 14 1] \\
5 & 91.43\%  & 99.54\%  & 0.0\%  & 95.52\% & [651 58; 3 0] \\
7 & 91.85\%  & 100.0\%  & 0.0\%  & 95.75\% & [654 58; 0 0] \\
10 & 91.85\%  & 100.0\%  & 0.0\%  & 95.75\% & [654 58; 0 0] \\
15 & 91.85\%  & 100.0\%  & 0.0\%  & 95.75\% & [654 58; 0 0] \

### Decision Tree

In [13]:
include("functions.jl")
dtParameters = Dict("modelType" => :DecisionTree, "maxDepth" => 1)

depths = [3, 5, 7, 10, 15, typemax(Int)]
for depth in depths
    dtParameters["maxDepth"] = depth
    metricsCV = (modelCrossValidation(dtParameters["modelType"], dtParameters, train_input, train_output, crossValidationIndexes))
    metricsCV["topology"] = depth

    generate_latex_table(metricsCV, false)

end

println("----------------------------------------------------------------")

for depth in depths
    dtParameters["maxDepth"] = depth
    metrics = createAndTrainFinalModel(dtParameters["modelType"], dtParameters, train_input, train_output, test_input, test_output)
    metrics["topology"] = depth

    generate_latex_table(metrics, true)

end


3 & 93.22\% \textit{(0.01)} & 99.47\% \textit{(0.01)} & 0.0\% \textit{(0.0)} & 96.49\% \textit{(0.0)} \\
5 & 92.8\% \textit{(0.01)} & 98.99\% \textit{(0.01)} & 0.56\% \textit{(0.02)} & 96.26\% \textit{(0.0)} \\
7 & 91.88\% \textit{(0.01)} & 97.94\% \textit{(0.01)} & 1.7\% \textit{(0.03)} & 95.76\% \textit{(0.01)} \\
10 & 90.37\% \textit{(0.02)} & 96.14\% \textit{(0.02)} & 4.48\% \textit{(0.04)} & 94.92\% \textit{(0.01)} \\
15 & 88.33\% \textit{(0.02)} & 93.93\% \textit{(0.02)} & 5.03\% \textit{(0.05)} & 93.77\% \textit{(0.01)} \\
9223372036854775807 & 87.07\% \textit{(0.02)} & 92.5\% \textit{(0.02)} & 6.21\% \textit{(0.06)} & 93.05\% \textit{(0.01)} \\
----------------------------------------------------------------
3 & 83.43\%  & 90.83\%  & 0.0\%  & 90.96\% & [594 58; 60 0] \\
5 & 84.55\%  & 91.74\%  & 3.45\%  & 91.6\% & [600 56; 54 2] \\
7 & 84.55\%  & 91.9\%  & 1.72\%  & 91.62\% & [601 57; 53 1] \\
10 & 83.85\%  & 91.13\%  & 1.72\%  & 91.2\% & [596 57; 58 1] \\
15 & 83.71\%  & 90.98

### SVM

In [14]:
include("functions.jl")
svmParameters = Dict("modelType" => :SVM, "C" => 1, "kernel" => "linear", "degree" => 3, "gamma" => "scale")

svms = [
    ("rbf", 0.1),
    ("rbf", 1.0),
    ("rbf", 10.0),
    ("poly", 0.1),
    ("poly", 1.0),
    ("linear", 0.1),
    ("linear", 1.0),
    ("linear", 10.0),
]

for (kernel, C) in svms
    svmParameters["kernel"] = kernel
    svmParameters["C"] = C
    metricsCV = (modelCrossValidation(svmParameters["modelType"], svmParameters, train_input, train_output, crossValidationIndexes))
    metricsCV["topology"] = kernel * string(C)

    generate_latex_table(metricsCV, false)

end

println("----------------------------------------------------------------")

for (kernel, C) in svms
    svmParameters["kernel"] = kernel
    svmParameters["C"] = C
    metrics = createAndTrainFinalModel(svmParameters["modelType"], svmParameters, train_input, train_output, test_input, test_output)
    metrics["topology"] = kernel * string(C)

    generate_latex_table(metrics, true)

end


rbf0.1 & 93.71\% \textit{(0.0)} & 100.0\% \textit{(0.0)} & 0.0\% \textit{(0.0)} & 96.75\% \textit{(0.0)} \\
rbf1.0 & 93.71\% \textit{(0.0)} & 100.0\% \textit{(0.0)} & 0.0\% \textit{(0.0)} & 96.75\% \textit{(0.0)} \\
rbf10.0 & 93.64\% \textit{(0.0)} & 99.93\% \textit{(0.0)} & 0.0\% \textit{(0.0)} & 96.71\% \textit{(0.0)} \\
poly0.1 & 93.71\% \textit{(0.0)} & 100.0\% \textit{(0.0)} & 0.0\% \textit{(0.0)} & 96.75\% \textit{(0.0)} \\
poly1.0 & 93.67\% \textit{(0.0)} & 99.96\% \textit{(0.0)} & 0.0\% \textit{(0.0)} & 96.73\% \textit{(0.0)} \\
linear0.1 & 93.71\% \textit{(0.0)} & 100.0\% \textit{(0.0)} & 0.0\% \textit{(0.0)} & 96.75\% \textit{(0.0)} \\
linear1.0 & 93.71\% \textit{(0.0)} & 100.0\% \textit{(0.0)} & 0.0\% \textit{(0.0)} & 96.75\% \textit{(0.0)} \\
linear10.0 & 93.71\% \textit{(0.0)} & 100.0\% \textit{(0.0)} & 0.0\% \textit{(0.0)} & 96.75\% \textit{(0.0)} \\
----------------------------------------------------------------
rbf0.1 & 91.85\%  & 100.0\%  & 0.0\%  & 95.75\% & [654 58;

### ANN

In [15]:
include("functions.jl")

#topologies = [[20], [40], [80], [100]]
topologies = [[60, 120], [80, 50], [80, 100], [100, 40]]
annParameters = Dict("modelType" => :ANN, "maxEpochs" => 200,
    "learningRate" => 0.01, "maxEpochsVal" => 30,
    "repetitions" => 30, "validationRatio" => 0.1,
    "transferFunctions" => fill(σ, 2))

for topology in topologies
    annParameters["topology"] = topology
    metricsCV = modelCrossValidation(annParameters["modelType"], annParameters, train_input, train_output, crossValidationIndexes)
    metricsCV["topology"] = topology 

    generate_latex_table(metricsCV, false)
end

for topology in topologies
    annParameters["topology"] = topology
    metrics = createAndTrainFinalModel(annParameters["modelType"], annParameters, train_input, train_output, test_input, test_output)
    metrics["topology"] = topology 

    generate_latex_table(metrics, true)
end

[60, 120] & 93.71\% \textit{(0.0)} & 100.0\% \textit{(0.0)} & 0.0\% \textit{(0.0)} & 96.75\% \textit{(0.0)} \\
[80, 50] & 93.71\% \textit{(0.0)} & 100.0\% \textit{(0.0)} & 0.0\% \textit{(0.0)} & 96.75\% \textit{(0.0)} \\
[80, 100] & 93.71\% \textit{(0.0)} & 100.0\% \textit{(0.0)} & 0.0\% \textit{(0.0)} & 96.75\% \textit{(0.0)} \\
[100, 40] & 93.71\% \textit{(0.0)} & 100.0\% \textit{(0.0)} & 0.0\% \textit{(0.0)} & 96.75\% \textit{(0.0)} \\
[60, 120] & 91.85\%  & 100.0\%  & 0.0\%  & 95.75\% & [654 58; 0 0] \\
[80, 50] & 91.85\%  & 100.0\%  & 0.0\%  & 95.75\% & [654 58; 0 0] \\
[80, 100] & 91.85\%  & 100.0\%  & 0.0\%  & 95.75\% & [654 58; 0 0] \\
[100, 40] & 91.85\%  & 100.0\%  & 0.0\%  & 95.75\% & [654 58; 0 0] \\


### Ensembles

In [21]:
@sk_import ensemble:StackingClassifier


PyObject <class 'sklearn.ensemble._stacking.StackingClassifier'>

In [25]:
include("functions.jl")

dtParameters = Dict("maxDepth" => 8)
knnParameters = Dict("numNeighboors" => 7)
svmParameters = Dict("kernel" => "rbf", "C" => 0.1, "degree" => 3, "gamma" => 2)
Random.seed!(42)

trainClassEnsemble([:DecisionTree, :kNN, :SVM], [dtParameters, knnParameters, svmParameters], (train_input, train_output), crossValidationIndexes)


Dict{String, Any} with 10 entries:
  "std_recall"      => 0.0856939
  "std_specificity" => 0.0825274
  "std_precision"   => 0.00707432
  "f1_score"        => 0.698131
  "specificity"     => 0.384967
  "std_accuracy"    => 0.0764194
  "std_f1_score"    => 0.0690458
  "accuracy"        => 0.551542
  "recall"          => 0.56265
  "precision"       => 0.931225

## Third approach: multiclass classification

In [11]:
Random.seed!(42)

train_indexes, test_indexes = holdOut(size(input_data, 1), 0.2)

train_input = convert(Array{Float32, 2}, input_data[train_indexes, :])
train_output = output_data[train_indexes]

normalizationParameters = calculateMinMaxNormalizationParameters(train_input)

normalizeMinMax!(train_input, normalizationParameters)

test_input = convert(Array{Float32, 2}, input_data[test_indexes, :])
test_output = output_data[test_indexes]

normalizeMinMax!(test_input, normalizationParameters)

@assert size(test_input, 1) == size(test_output, 1)
@assert size(train_input, 1) == size(train_output, 1)

In [15]:
include("functions.jl")
dtParameters = Dict("modelType" => :DecisionTree, "maxDepth" => 1)

depths = [3, 5, 7, 10, 15, typemax(Int)]
for depth in depths
    dtParameters["maxDepth"] = depth
    metricsCV = (modelCrossValidation(dtParameters["modelType"], dtParameters, train_input, train_output, crossValidationIndexes))
    metricsCV["topology"] = depth

    generate_latex_table(metricsCV, false)

end

println("----------------------------------------------------------------")

for depth in depths
    dtParameters["maxDepth"] = depth
    metrics = createAndTrainFinalModel(dtParameters["modelType"], dtParameters, train_input, train_output, test_input, test_output)
    metrics["topology"] = depth

    generate_latex_table(metrics, true)

end


3 & 40.81\% \textit{(0.04)} & 40.81\% \textit{(0.04)} & 61.26\% \textit{(0.03)} & 30.85\% \textit{(0.04)} \\
5 & 41.76\% \textit{(0.03)} & 41.76\% \textit{(0.03)} & 63.33\% \textit{(0.03)} & 34.26\% \textit{(0.05)} \\
7 & 40.88\% \textit{(0.03)} & 40.88\% \textit{(0.03)} & 64.42\% \textit{(0.02)} & 35.53\% \textit{(0.03)} \\
10 & 41.41\% \textit{(0.03)} & 41.41\% \textit{(0.03)} & 66.5\% \textit{(0.02)} & 37.61\% \textit{(0.03)} \\
15 & 40.56\% \textit{(0.03)} & 40.56\% \textit{(0.03)} & 68.93\% \textit{(0.03)} & 38.95\% \textit{(0.03)} \\
9223372036854775807 & 39.23\% \textit{(0.02)} & 39.23\% \textit{(0.02)} & 71.47\% \textit{(0.02)} & 39.21\% \textit{(0.02)} \\
----------------------------------------------------------------
3 & 40.59\%  & 40.59\%  & 61.36\%  & 32.37\% & [32 191 0 9; 43 246 0 10; 16 38 0 4; 21 91 0 11] \\
5 & 44.8\%  & 44.8\%  & 64.75\%  & 37.39\% & [65 161 0 6; 46 248 1 4; 17 38 0 3; 46 71 0 6] \\
7 & 43.4\%  & 43.4\%  & 67.15\%  & 40.0\% & [92 129 0 11; 87 195 1 1

In [16]:
@sk_import decomposition:PCA


PyObject <class 'sklearn.decomposition._pca.PCA'>

In [17]:

# pcas = 1:20:4
pca = PCA(2)

#Ajust the matrix acording to the train data
fit!(pca, train_input)

#Once it is ajusted it can be used to transform the data
pca_train = pca.transform(train_input)
pca_test = pca.transform(test_input)

@assert (size(train_input)[1],2) == size(pca_train)
@assert (size(test_input)[1],2) == size(pca_test)

In [24]:
pcas = [2, 6, 10, 15, 20, 25, 30]

7-element Vector{Int64}:
  2
  6
 10
 15
 20
 25
 30

In [25]:
include("functions.jl")
dtParameters = Dict("modelType" => :DecisionTree, "maxDepth" => 1)

for pca_value in pcas
    pca = PCA(pca_value)

    #Ajust the matrix acording to the train data
    fit!(pca, train_input)

    #Once it is ajusted it can be used to transform the data
    pca_train = pca.transform(train_input)
    pca_test = pca.transform(test_input)

    println("---------------------------$pca_value-------------------------------------")


    depths = [3, 5, 7, 10, 15, typemax(Int)]
    for depth in depths
        dtParameters["maxDepth"] = depth
        metricsCV = (modelCrossValidation(dtParameters["modelType"], dtParameters, pca_train, train_output, crossValidationIndexes))
        metricsCV["topology"] = depth

        generate_latex_table(metricsCV, false)

    end

    println("----------------------------------------------------------------")

    for depth in depths
        dtParameters["maxDepth"] = depth
        metrics = createAndTrainFinalModel(dtParameters["modelType"], dtParameters, pca_train, train_output, pca_test, test_output)
        metrics["topology"] = depth

        generate_latex_table(metrics, true)

    end
end

---------------------------2-------------------------------------
3 & 39.97\% \textit{(0.03)} & 39.97\% \textit{(0.03)} & 58.94\% \textit{(0.03)} & 25.67\% \textit{(0.04)} \\
5 & 39.93\% \textit{(0.04)} & 39.93\% \textit{(0.04)} & 59.9\% \textit{(0.03)} & 28.78\% \textit{(0.04)} \\
7 & 38.53\% \textit{(0.03)} & 38.53\% \textit{(0.03)} & 61.07\% \textit{(0.02)} & 30.63\% \textit{(0.04)} \\
10 & 38.7\% \textit{(0.04)} & 38.7\% \textit{(0.04)} & 63.19\% \textit{(0.02)} & 33.62\% \textit{(0.03)} \\
15 & 36.59\% \textit{(0.03)} & 36.59\% \textit{(0.03)} & 65.51\% \textit{(0.02)} & 34.52\% \textit{(0.03)} \\
9223372036854775807 & 34.83\% \textit{(0.02)} & 34.83\% \textit{(0.02)} & 70.0\% \textit{(0.02)} & 35.01\% \textit{(0.02)} \\
----------------------------------------------------------------
3 & 41.99\%  & 41.99\%  & 58.01\%  & 24.84\% & [0 232 0 0; 0 299 0 0; 0 58 0 0; 0 123 0 0] \\
5 & 42.42\%  & 42.42\%  & 63.85\%  & 36.35\% & [87 144 0 1; 84 211 0 4; 20 36 1 1; 47 73 0 3] \\
7 & 41.1